<a href="https://colab.research.google.com/github/piaosan78/AIFFEL_Quest/blob/master/Exploration_CR5/Chapter_12/MLOps_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 구글 드라이브 임포트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 모델 저장할 폴더 생성 및 케라스 튜너 설치

In [2]:
!mkdir /content/drive/MyDrive/mlops_cifar10
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 49.8 MB/s eta 0:00:00


# 라이브러리 임포트

In [17]:
import tensorflow as tf
import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import os

# 데이터셋 로드

In [19]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# X 1~0 사이로 정규화, y훈련 원핫 인코딩

In [20]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
y_train.shape

(50000, 10)

# 검증 데이터 분리

In [21]:
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)
print(X_train.shape)

(40000, 32, 32, 3)


# 케라스 튜너 구성

In [22]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        model.fit(X, y, batch_size=trial.hyperparameters.Choice(
            'batch_size', [16, 32]), **fit_kwargs)


        X_val, y_val = validation_data
        eval_scores = model.evaluate(X_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}

# 모델 구성

In [23]:
def build_cifar10_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(32, 32, 3)))

    for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
        model.add(tf.keras.layers.Conv2D(hp.Int(f'conv_units_{i}', min_value=32, max_value=128, step=32),
                                         (3,3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))

    for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
        model.add(tf.keras.layers.Conv2D(hp.Int(f'conv_units_{i}', min_value=32, max_value=128, step=32),
                                         (3,3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))

    for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
        model.add(tf.keras.layers.Conv2D(hp.Int(f'conv_units_{i}', min_value=32, max_value=128, step=32),
                                         (3,3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# 모델 학습

In [27]:
class CustomTuner(kt.Tuner):

    def run_trial(self, trial, *args, **kwargs):
        # You can add additional customization here
        return super(CustomTuner, self).run_trial(trial, *args, **kwargs)

# Set up the custom tuner
my_keras_tuner = CustomTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('val_accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_cifar10_model,
    overwrite=True,
    project_name='cifar10_keras_tuner'
)

# Now, the search should work without errors
my_keras_tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=10)


Trial 10 Complete [00h 00m 00s]

Best val_accuracy So Far: 0.6919000148773193
Total elapsed time: 00h 06m 03s


# 모델 구조 확인

In [28]:
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=1)[0]
model = build_cifar10_model(best_hps)
model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 30, 30, 64)        1792      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 15, 15, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 13, 13, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 6, 6, 128)         0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 4, 4, 32)          36896     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 2, 2, 32)         

# 최적화 모델 학습

In [29]:
model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val))
model.evaluate(x_test, y_test)

Epoch 1/20
1250/1250 [==============================] - 7s 4ms/step - loss: 1.6529 - accuracy: 0.3900 - val_loss: 1.4282 - val_accuracy: 0.4790
Epoch 2/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.2550 - accuracy: 0.5520 - val_loss: 1.2335 - val_accuracy: 0.5599
Epoch 3/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.1147 - accuracy: 0.6049 - val_loss: 1.1424 - val_accuracy: 0.5930
Epoch 4/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.0126 - accuracy: 0.6427 - val_loss: 1.0424 - val_accuracy: 0.6295
Epoch 5/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.9445 - accuracy: 0.6677 - val_loss: 1.0205 - val_accuracy: 0.6390
Epoch 6/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.8837 - accuracy: 0.6891 - val_loss: 0.9722 - val_accuracy: 0.6577
Epoch 7/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.8281 - accuracy: 0.7101 - val_loss: 0.9462 - val_accuracy:

[0.9975835084915161, 0.7006999850273132]

# 모델 평가

In [31]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.9976 - accuracy: 0.7007


[0.9975835084915161, 0.7006999850273132]

#

# 최적 모델 저장

In [32]:
model.save('/content/drive/MyDrive/mlops_cifar10/best_model/1')

# 최적 모델 불러오기

In [34]:
load_path = '/content/drive/MyDrive/mlops_cifar10/best_model/1'
best_model = tf.keras.models.load_model(load_path)

# 최적 모델 구성 확인

In [35]:
best_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 30, 30, 64)        1792      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 15, 15, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 13, 13, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 6, 6, 128)         0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 4, 4, 32)          36896     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 2, 2, 32)         

In [36]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [37]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [38]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)

signatures = interpreter.get_signature_list()
print(signatures)

{'serving_default': {'inputs': ['input_2'], 'outputs': ['dense_1']}}


In [39]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite